In [1]:

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import spams
import numpy as np
import skimage 
import tensorflow as tf
import math as m
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import layers
from mxnet import nd
import mxnet as mx
from tensorflow import keras
from tensorflow.keras import losses
import pandas as pd

2023-06-17 16:58:45.143621: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
import pickle
num_classes = 11
# 加载数据''
with open('/home/xipeng/FLcode/Organa/my_data_0.5.pkl', 'rb') as f:
    data_x , data_y,x_test,y_test  = pickle.load(f)
x_train =  data_x 
y_train =  data_y
for i in range(30):
    df = pd.DataFrame(data_y[i])
    label_counts = df.sum(axis=0)
    label_counts = label_counts[label_counts > 0]
# 提取标签和数量
    labels = np.array(label_counts.index, dtype=np.uint8)
    counts = np.array(label_counts.values)
# 打印结果
    print(f" {i}: ",(labels, counts))

 0:  (array([3, 4, 6, 7, 8], dtype=uint8), array([ 97., 238., 353., 236., 228.], dtype=float32))
 1:  (array([ 2,  3,  4,  7, 10], dtype=uint8), array([111., 105., 315., 325., 296.], dtype=float32))
 2:  (array([2, 3, 4, 7, 8], dtype=uint8), array([103., 106., 298., 337., 308.], dtype=float32))
 3:  (array([1, 2, 3, 5, 6], dtype=uint8), array([111., 116., 116., 309., 500.], dtype=float32))
 4:  (array([1, 2, 3, 5, 9], dtype=uint8), array([123., 155., 160., 418., 296.], dtype=float32))
 5:  (array([ 1,  3,  7,  8, 10], dtype=uint8), array([106., 120., 330., 313., 283.], dtype=float32))
 6:  (array([ 0,  6,  7,  8, 10], dtype=uint8), array([113., 384., 239., 220., 196.], dtype=float32))
 7:  (array([0, 2, 6, 7, 9], dtype=uint8), array([141., 118., 402., 264., 227.], dtype=float32))
 8:  (array([0, 1, 2, 4, 9], dtype=uint8), array([185., 130., 138., 375., 324.], dtype=float32))
 9:  (array([ 1,  4,  7,  8, 10], dtype=uint8), array([ 97., 274., 256., 280., 245.], dtype=float32))
 10:  (arr

In [4]:
def  set_model():
    model=Sequential()
        # 定义顺序模型           
    model.add(Convolution2D(
        input_shape = (28,28,1),
        filters = 8,
        kernel_size = 3,
        strides = 1,
        padding = 'same',
        activation = 'relu',
    ))
    # 第一个池化层
    model.add(MaxPooling2D(
        pool_size = 2,
        strides = 2,
        padding = 'same',
    ))
    # 第二个卷积层
    model.add(Convolution2D(16,3,strides=1,padding='same',activation='relu'))
    # 第二个池化层
    model.add(MaxPooling2D(2,2,'same'))                                    
                  
       # 第二个卷积层      
    # 把第三个池化层的输出扁平化为1维
    model.add(layers.Flatten())
    # 第一个全连接层
    model.add(Dense(256,activation='relu'))
    # Dropout
    model.add(Dropout(0.5)) 
    # 第二个全连接层
    model.add(Dense(num_classes,activation='softmax'))

    # 定义优化器
    adam = Adam(lr = 5e-4)        
        
    # 定义优化器,loss function,训练过程中计算准确率
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

        #print(model_d[i])
    return model

In [5]:
def compute_square_l2_distance(grads1, grads2):
    distance = 0
    for g1, g2 in zip(grads1, grads2):
        distance += np.linalg.norm(np.array(g1) - np.array(g2)) ** 2
    return distance

def compute_distances(grads):
    n = len(grads)
    distances = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            distance = compute_square_l2_distance(grads[i], grads[j])
            distances[i, j] = distance
            distances[j, i] = distance

    return distances
def krum(grads, f):
    """
    Krum algorithm for robust gradient aggregation.
    
    :param grads: List of gradients from participating devices.
    :param f: The number of Byzantine (malicious or faulty) devices to tolerate.
    :return: The aggregated gradient.
    """
    n = len(grads)
    distances = compute_distances(grads)
    min_krum_scores = float("inf")
    selected_device_index = -1

    for i in range(n):
        krum_scores = np.partition(distances[i], f)[:f]
        krum_score = np.sum(krum_scores)

        if krum_score < min_krum_scores:
            min_krum_scores = krum_score
            selected_device_index = i
            print(i)
    return grads[selected_device_index]


In [6]:
def grad_all(model):
    gradients = []
    for i in range (n):
        for epoch in range(3):
            with tf.GradientTape() as tape:
                # Get the model's predictions
                predictions = model[i](x_train[i])
            # Compute the loss using the true labels and the predictions
                loss_function = losses.get(model[i].loss)
                loss = loss_function(y_train[i], predictions)
            
        gradients.append(tape.gradient(loss, model[i].trainable_variables))  
    return gradients

In [7]:
def sc(gradients,t):
    for i in range (t):
        for j in range (8):
            gradients[i][j] = 1.5*gradients[i][j]
    return gradients

In [8]:


def noise(gradients,t):
    for i in range (t):
        for j in range (8):
            gradients_np = gradients[i][j].numpy() 
            noisy_gradients_np = skimage.util.random_noise(gradients_np, mode="gaussian", var=0.2, clip=True)
            gradients[i][j] = tf.convert_to_tensor(noisy_gradients_np, dtype=tf.float32)  # Convert back to tensor with dtype=float32
    return gradients


In [9]:
# def grad_all(model):
#     gradients = []
#     for i in range (n):
#         for epoch in range(3):
#             with tf.GradientTape() as tape:
#                 # Get the model's predictions
#                 predictions = model[i](x_train[i])
#             # Compute the loss using the true labels and the predictions
#                 loss_function = losses.get(model[i].loss)
#                 loss = loss_function(y_train[i], predictions)
            
#         gradients.append(tape.gradient(loss, model[i].trainable_variables))  
#     return gradients

In [10]:
import scipy
from scipy.io import loadmat

n= 30;

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)

global_model = set_model()


loss_9=[]
t=9
for s in range (100):
    gradients = grad_all(model_list)
    ##
    #可以考虑在这中间加入攻击  先看一下gradients的shape，然后看怎么加入攻击
    gradients = noise(gradients,t)
    ##
    selected_gradient = krum(gradients,10) 
    grads_and_vars = zip(selected_gradient, global_model.trainable_variables)
    global_model.optimizer.apply_gradients(grads_and_vars)
    loss,accuracy = global_model.evaluate(x_test,y_test,verbose=1)
    loss_9.append(loss)
    print(s)
    for j in range(len(model_list)):
        model_list[j].set_weights(global_model.get_weights())
scipy.io.savemat('loss_9_1.mat', mdict={'loss_9': loss_9}) 

2023-06-17 16:58:49.847272: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-06-17 16:58:49.881300: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-06-17 16:58:49.881358: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-06-17 16:58:49.881438: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-06-17 16:58:49.881472: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-06-17 16:58:49.881507: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

0
6
8
556/556 [==============================] - 8s 14ms/step - loss: 2.4033 - accuracy: 0.1146
0
0
2
556/556 [==============================] - 2s 3ms/step - loss: 2.3998 - accuracy: 0.1078
1
0
2
6
556/556 [==============================] - 2s 3ms/step - loss: 2.3840 - accuracy: 0.1083
2
0
1
2
5
6
556/556 [==============================] - 2s 3ms/step - loss: 2.3652 - accuracy: 0.1160
3
0
1
2
5
6
556/556 [==============================] - 2s 3ms/step - loss: 2.3484 - accuracy: 0.1347
4
0
1
556/556 [==============================] - 2s 3ms/step - loss: 2.3388 - accuracy: 0.1696
5
0
1
556/556 [==============================] - 2s 3ms/step - loss: 2.3343 - accuracy: 0.1855
6
0
1
2
556/556 [==============================] - 2s 3ms/step - loss: 2.3391 - accuracy: 0.1571
7
0
1
5
556/556 [==============================] - 2s 3ms/step - loss: 2.3501 - accuracy: 0.1379
8
0
1
5
556/556 [==============================] - 2s 3ms/step - loss: 2.3673 - accuracy: 0.1283
9
0
1
556/556 [==============

In [11]:
import scipy
from scipy.io import loadmat

n= 30;

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)

global_model = set_model()


loss_6=[]
t=6
for s in range (100):
    gradients = grad_all(model_list)
    ##
    #可以考虑在这中间加入攻击  先看一下gradients的shape，然后看怎么加入攻击
    gradients = noise(gradients,t)
    ##
    selected_gradient = krum(gradients,10) 
    grads_and_vars = zip(selected_gradient, global_model.trainable_variables)
    global_model.optimizer.apply_gradients(grads_and_vars)
    loss,accuracy = global_model.evaluate(x_test,y_test,verbose=1)
    loss_6.append(loss)
    print(s)
    for j in range(len(model_list)):
        model_list[j].set_weights(global_model.get_weights())
scipy.io.savemat('loss_6_1.mat', mdict={'loss_6': loss_6}) 

0
2
3
556/556 [==============================] - 2s 4ms/step - loss: 2.4696 - accuracy: 0.0429
0
0
1
5
556/556 [==============================] - 2s 4ms/step - loss: 2.4678 - accuracy: 0.0430
1
0
1
2
5
556/556 [==============================] - 2s 4ms/step - loss: 2.4735 - accuracy: 0.0440
2
0
1
2
5
556/556 [==============================] - 2s 4ms/step - loss: 2.4903 - accuracy: 0.1024
3
0
1
2
5
556/556 [==============================] - 2s 4ms/step - loss: 2.5198 - accuracy: 0.2296
4
0
1
5
556/556 [==============================] - 2s 4ms/step - loss: 2.5654 - accuracy: 0.2661
5
0
5
556/556 [==============================] - 2s 4ms/step - loss: 2.6302 - accuracy: 0.2714
6
0
556/556 [==============================] - 2s 4ms/step - loss: 2.6760 - accuracy: 0.2592
7
0
556/556 [==============================] - 2s 4ms/step - loss: 2.7023 - accuracy: 0.2237
8
0
556/556 [==============================] - 2s 4ms/step - loss: 2.7104 - accuracy: 0.1881
9
0
556/556 [===========================

In [12]:

import scipy
from scipy.io import loadmat

n= 30;

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)

global_model = set_model()


loss_3=[]
t=3
for s in range (100):
    gradients = grad_all(model_list)
    ##
    #可以考虑在这中间加入攻击  先看一下gradients的shape，然后看怎么加入攻击
    gradients = noise(gradients,t)
    ##
    selected_gradient = krum(gradients,10) 
    grads_and_vars = zip(selected_gradient, global_model.trainable_variables)
    global_model.optimizer.apply_gradients(grads_and_vars)
    loss,accuracy = global_model.evaluate(x_test,y_test,verbose=1)
    loss_3.append(loss)
    print(s)
    for j in range(len(model_list)):
        model_list[j].set_weights(global_model.get_weights())
scipy.io.savemat('loss_3_1.mat', mdict={'loss_3': loss_3}) 


0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.4252 - accuracy: 0.1570
0
0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.4296 - accuracy: 0.1003
1
0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.4468 - accuracy: 0.1052
2
0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.4799 - accuracy: 0.1089
3
0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.5313 - accuracy: 0.1137
4
0
1
2
556/556 [==============================] - 2s 4ms/step - loss: 2.6023 - accuracy: 0.1178
5
0
2
556/556 [==============================] - 2s 4ms/step - loss: 2.6939 - accuracy: 0.1261
6
0
556/556 [==============================] - 2s 4ms/step - loss: 2.7714 - accuracy: 0.1357
7
0
556/556 [==============================] - 2s 4ms/step - loss: 2.8325 - accuracy: 0.1469
8
0
556/556 [==============================] - 2s 4ms/step - loss: 2.8747 - accuracy: 0.1574
9
0
6
556/556 [==============================]

In [13]:
for s in range (100):
    gradients = grad_all(model_list)
    ##
    #可以考虑在这中间加入攻击  先看一下gradients的shape，然后看怎么加入攻击
    gradients = noise(gradients,t)
    ##
    selected_gradient = krum(gradients,20) 
    grads_and_vars = zip(selected_gradient, global_model.trainable_variables)
    global_model.optimizer.apply_gradients(grads_and_vars)
    loss,accuracy = global_model.evaluate(x_test,y_test,verbose=1)
    loss_3.append(loss)
    print(s)
    for j in range(len(model_list)):
        model_list[j].set_weights(global_model.get_weights())
scipy.io.savemat('loss_3_1.mat', mdict={'loss_3': loss_3}) 


0
1
6
22
27
556/556 [==============================] - 2s 4ms/step - loss: 2.9311 - accuracy: 0.4450
0
0
1
9
22
27
556/556 [==============================] - 2s 4ms/step - loss: 2.8167 - accuracy: 0.4075
1
0
1
2
9
22
27
556/556 [==============================] - 2s 4ms/step - loss: 2.7018 - accuracy: 0.4025
2
0
1
2
9
27
556/556 [==============================] - 2s 4ms/step - loss: 2.5847 - accuracy: 0.4006
3
0
1
2
9
27
556/556 [==============================] - 2s 4ms/step - loss: 2.4679 - accuracy: 0.3998
4
0
1
2
9
27
556/556 [==============================] - 2s 4ms/step - loss: 2.3580 - accuracy: 0.3969
5
0
2
9
27
556/556 [==============================] - 2s 4ms/step - loss: 2.2624 - accuracy: 0.3926
6
0
2
556/556 [==============================] - 2s 4ms/step - loss: 2.1886 - accuracy: 0.3890
7
0
2
556/556 [==============================] - 2s 4ms/step - loss: 2.1321 - accuracy: 0.3868
8
0
2
556/556 [==============================] - 2s 4ms/step - loss: 2.0892 - accuracy: 0.3867
